# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install citipy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
#output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)



### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

lat_lngs

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

616

## Generate Cities List

In [4]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
city = cities

query_url = f"{base_url}appid={weather_api_key}&q={city}&units=imperial"

# Get weather data
weather_response = requests.get(query_url).json()
weather_response

{'coord': {'lon': 169.85, 'lat': -46.28},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 54,
  'feels_like': 52.34,
  'temp_min': 54,
  'temp_max': 54,
  'pressure': 1013,
  'humidity': 79},
 'visibility': 10000,
 'wind': {'speed': 1.99, 'deg': 0, 'gust': 5.99},
 'clouds': {'all': 100},
 'dt': 1608219071,
 'sys': {'type': 3,
  'id': 2000217,
  'country': 'NZ',
  'sunrise': 1608223423,
  'sunset': 1608280238},
 'timezone': 46800,
 'id': 2208248,
 'name': 'Kaitangata',
 'cod': 200}

In [5]:
weather_response["wind"]["speed"]

1.99

In [20]:
cities_results = []

for index,city in enumerate(cities):
    
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    query_url = f"{base_url}appid={weather_api_key}&q={city}"

    try:
        # Get weather data
        weather_response = requests.get(query_url).json()
        lon=weather_response["coord"]["lon"]
        lat=weather_response["coord"]["lat"]
        cities_results.append(weather_response)
    except:
        print(f"No City Data Found for {city}")

(cities_results[0])


No City Data Found for taolanaro
No City Data Found for palabuhanratu
No City Data Found for samalaeulu
No City Data Found for viligili
No City Data Found for tsihombe
No City Data Found for belushya guba
No City Data Found for jarjis
No City Data Found for babanusah
No City Data Found for karkaralinsk
No City Data Found for samusu
No City Data Found for amderma
No City Data Found for bargal


{'coord': {'lon': 169.85, 'lat': -46.28},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 284.82,
  'feels_like': 283.86,
  'temp_min': 284.82,
  'temp_max': 284.82,
  'pressure': 1013,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 0.89, 'deg': 0, 'gust': 3.13},
 'clouds': {'all': 100},
 'dt': 1608221551,
 'sys': {'type': 3,
  'id': 2000217,
  'country': 'NZ',
  'sunrise': 1608223423,
  'sunset': 1608280238},
 'timezone': 46800,
 'id': 2208248,
 'name': 'Kaitangata',
 'cod': 200}

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [54]:
city_data = []
for i,city in enumerate(cities_results):
    citytemp = city["main"]["temp"]
    citylat = city["coord"]["lat"]
    cityhum = city["main"]["humidity"]
    citycloud = city["clouds"]["all"]
    citywind = city["wind"]["speed"]
    
    city_data.append({"Temperature": citytemp,
    "Latitude": citylat,
    "Humidity" : cityhum,
    "Cloudiness" : citycloud,
    "Wind Speed" : citywind
    })
        
    
    

In [55]:
df = pd.DataFrame(city_data)

In [56]:
df.head()

,Temperature,Latitude,Humidity,Cloudiness,Wind Speed
0,284.82,-46.28,81,100,0.89
1,283.06,37.80,87,75,3.10
2,240.36,71.97,82,1,3.51
3,280.15,-54.80,70,75,17.50
4,260.32,69.70,91,33,5.50
...,...,...,...,...,...
83,288.15,-29.95,82,90,4.60
84,281.05,40.60,100,90,2.60
85,296.15,-24.87,88,0,7.70
86,294.41,-5.06,87,100,1.74


In [64]:
df_final =pd.DataFrame()
for i,city in enumerate(cities_results):
    citytemp = city["main"]["temp"]
    citylat = city["coord"]["lat"]
    cityhum = city["main"]["humidity"]
    citycloud = city["clouds"]["all"]
    citywind = city["wind"]["speed"]
    
    temp= pd.DataFrame({"Temperature": citytemp,
    "Latitude": [citylat],
    "Humidity" : [cityhum],
    "Cloudiness" : [citycloud],
    "Wind Speed" : [citywind]
    })

    if i==0:
        df_final=temp
    else:
        df.append(temp)

In [65]:
df

,Temperature,Latitude,Humidity,Cloudiness,Wind Speed
0,284.82,-46.28,81,100,0.89
1,283.06,37.80,87,75,3.10
2,240.36,71.97,82,1,3.51
3,280.15,-54.80,70,75,17.50
4,260.32,69.70,91,33,5.50
...,...,...,...,...,...
83,288.15,-29.95,82,90,4.60
84,281.05,40.60,100,90,2.60
85,296.15,-24.87,88,0,7.70
86,294.41,-5.06,87,100,1.74


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [9]:
#  Get the indices of cities that have humidity over 100%.


In [10]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression